## Example codes for converting surface and voxel model to different formats and extract information from loop gml file

 - The surface conversion codes assume that a directory of obj format surfaces already exsists, which will exsist if using any of the example notebooks, and the code can be run directly from this notebook simply by updating the file paths
 - The voxel conversion codes need to be pasted at the end of the one of the Example notebooks as it relies on having access to a LoopStructural model object.
 - The loop.gml file combines all the disparate outputs from map2loop into a single graph which may be queried to retreive the indiivdual satasets, or to analyse topology

## Convert geotif of dtm to obj format mesh

In [ ]:
from map2loop.m2l_utils import save_dtm_mesh
obj_path_dir='D:/Dropbox/1_Jupyter_notebooks/Peak_District/Peak_District_out_07/dtm/'   # directory of obj surfaces
dtm_path='D:/Dropbox/1_Jupyter_notebooks/Peak_District/Peak_District_out_07/dtm/'  # directory of existing geotif dtm
save_dtm_mesh(dtm_path,obj_path_dir)


### GeoscienceAnalyst geohy5 format surfaces from directory of obj files

In [ ]:
#code to parse a directory of .obj files and combine them into
#a single geoh5 file that GeoscienceAnalyst can read
#Requires installation of https://github.com/MiraGeoscience/geoh5py 
#(note .geoh5 meshes are 0 based)
#Run this code after obj files have been created (see Example notebooks)

import pandas as pd
from os import listdir
import os
from os.path import isfile, join
from pathlib import Path
import numpy as np
from geoh5py.objects import BlockModel
from geoh5py.workspace import Workspace
from geoh5py.objects import Surface
import omf

def hextofloats(h):
    '''Takes a hex rgb string (e.g. #ffffff) and returns an RGB tuple (float, float, float).'''
    return tuple(int(h[i:i + 2], 16) / 255. for i in (1, 3, 5))  # skip '#'

def geoh5_create_surface_data(obj_path_dir,colour_path):

    h5file_path = obj_path_dir+"/loop.geoh5"

    workspace = Workspace(h5file_path)
    onlyfiles = [f for f in listdir(obj_path_dir) if isfile(join(obj_path_dir, f))]
    colour_index=0
    all_sorts = pd.read_csv(os.path.join(colour_path, 'all_sorts_clean.csv'), ",")
    all_sorts=all_sorts.set_index('code')
    colour_map=open(obj_path_dir+'/loop_colour_map.clr','w')
    colour_map.write('{\tStart\tRed\tGreen\tBlue\t}\n')

    for file in onlyfiles:
        if ('.obj' in file):
            obj=pd.read_csv(obj_path_dir+'/'+file,' ',names=["code","X","Y","Z"])
            indices=obj[obj['code']=='f']
            vertices=obj[obj['code']=='v']
            vertices=vertices.drop(['code'], axis=1)
            indices=indices[list("XYZ")].astype(int)
            i=indices.to_numpy()-1
            v=vertices.to_numpy()
            if(len(i)>0 and len(v)>0):
                # Create a geoh5 surface
                surface = Surface.create(
                          workspace, name=file.replace('.obj',''), vertices=v, cells=i
                            )
                if('Fault_' in file or 'dtm' in file):
                    colours=np.ones(surface.n_cells)*99
                else:
                    colours=np.ones(surface.n_cells)*colour_index
                    rgb=hextofloats(all_sorts.loc[file.replace('.obj','')]['colour'])
                    colour_map.write('{}\t{}\t{}\t{}\n'.format(colour_index,rgb[0],rgb[1],rgb[2]))
                    colour_index=colour_index+1
                    

                surface.add_data({
                    "colour_index": {
                        "association":"CELL",
                        "values": colours
                    }
                })
                
                workspace.save_entity(surface)
                workspace.finalize()
                
    colour_map.write('{}\t{}\t{}\t{}\n'.format(99,1,1,1))
    colour_map.close()
    print("colour map saved as:",obj_path_dir+'/loop_colour_map.clr')

obj_path_dir= 'C:/Users/mark/Desktop/july_models/model5_m/model5_m/'   # directory of existing obj surfaces
colour_path='C:/Users/mark/Desktop/july_models/model5_m/tmp/'
geoh5_create_surface_data(obj_path_dir,colour_path)


### dxf format surfaces from directory of obj files

In [ ]:
#code to parse a directory of .obj files and combine them into
#a single 3D DXF file that Micromine, Datamine, Vulcan... can read
#(note .dxf meshes are 1 based)
#Run this code after obj files have been created (see Example notebooks)

import pandas as pd
from os import listdir
from os.path import isfile, join
import numpy as np

def dxf_create_surface_data(obj_path_dir):


    onlyfiles = [f for f in listdir(obj_path_dir) if isfile(join(obj_path_dir, f))]
    
    fout = open(obj_path_dir+'/loop.dxf', "w")
    fout.write("  0\nSECTION\n  2\nENTITIES\n")

    for file in onlyfiles:
        if ('.obj' in file):
            obj=pd.read_csv(obj_path_dir+'/'+file,' ',names=["code","X","Y","Z"])
            indices=obj[obj['code']=='f']
            vertices=obj[obj['code']=='v']
            vertices=vertices.drop(['code'], axis=1)
            indices=indices[list("XYZ")].astype(int)
            i=indices.to_numpy()-1
            v=vertices.to_numpy()
           
            if(len(i)>0 and len(v)>0):
                for ind in range(0, i.shape[0]):
                    triangle="  0\n3DFACE\n  8\n{}\n  10\n{}\n  20\n{}\n  30\n{}\n  11\n{}\n  21\n{}\n  31\n{}\n  12\n{}\n  22\n{}\n  32\n{}\n  13\n{}\n  23\n{}\n  33\n{}\n".format(
                                file.replace('.obj',''),v[i[ind,0],0], v[i[ind,0],1], v[i[ind,0],2]
                                                        ,v[i[ind,1],0], v[i[ind,1],1], v[i[ind,1],2]
                                                        ,v[i[ind,2],0], v[i[ind,2],1], v[i[ind,2],2]
                                                        ,v[i[ind,2],0], v[i[ind,2],1], v[i[ind,2],2])
                    fout.write(triangle)
  
    fout.write("0\nENDSEC\n  0\nEOF\n")
    fout.write("END\n")
    fout.close()

            
#obj_path_dir= './Example2/vtkleaflet_2021-07-16-09-43/'   # directory of existing obj surfaces
dxf_create_surface_data(obj_path_dir)

### Gocad ts format surfaces from directory of obj files

In [ ]:
#code to parse a directory of .obj files and combine them into
#a set of Gocad .ts files that GeoscienceAnalyst and Gocad can read
#(note .ts meshes are 1 based)
#Run this code after obj files have been created (see Example notebooks)

import pandas as pd
from os import listdir
from os.path import isfile, join
import numpy as np

def gocad_create_surface_data(obj_path_dir):


    onlyfiles = [f for f in listdir(obj_path_dir) if isfile(join(obj_path_dir, f))]
    
    for file in onlyfiles:
        if ('.obj' in file):
            obj=pd.read_csv(obj_path_dir+'/'+file,' ',names=["code","X","Y","Z"])
            indices=obj[obj['code']=='f']
            vertices=obj[obj['code']=='v']
            vertices=vertices.drop(['code'], axis=1)
            indices=indices[list("XYZ")].astype(int)
            i=indices.to_numpy()
            v=vertices.to_numpy()
            
            if(len(i)>0 and len(v)>0): 
                fout = open(obj_path_dir+'/'+file.replace('.obj','')+'.ts', "w")
                #fout.write("GOCAD Tsurf 1 \nHEADER {\nname:" + file.replace('.obj','') + "\n}\nTFACE\n")
                fout.write("GOCAD TSurf 1\nHEADER {\nname:"+
                        file.replace('.obj','')+
                        "\n}\nTFACE\n")

                for ind in range(0, v.shape[0]):
                    fout.write("PVRTX %d %f %f %f\n" % (ind + 1, v[ind,0], v[ind,1], v[ind,2]))

                for ind in range(0,i.shape[0]):
                    fout.write("TRGL %d %d %d\n" % (i[ind,0], i[ind,1], i[ind,2]))


                fout.write("END\n")
                fout.close()

            
#obj_path_dir= './Example2/vtkleaflet_2021-07-16-09-43/'   # directory of existing obj surfaces
gocad_create_surface_data(obj_path_dir)

### Open Mining Format omf format surfaces from directory of obj files

In [ ]:
#code to parse a directory of .obj files and combine them into
#a single 3D OMF file that Micromine, Datamine, Vulcan... can read
#(note .omf meshes are 1 based) 
# requires pip install omf (and omfvista and  ipyvtklink if you want to visualise them)
# https://omf.readthedocs.io/en/latest/content/examples.html
# https://gmggroup.org/projects/data-exchange-for-mine-software/
# https://github.com/gmggroup/omf
#Run this code after obj files have been created (see Example notebooks)

from os import listdir
from os.path import isfile, join
import numpy as np
import omf
import random
import pandas as pd
import copy
def hextoints(h):
    '''Takes a hex rgb string (e.g. #ffffff) and returns an RGB tuple (float, float, float).'''
    return tuple(int(h[i:i + 2], 16) for i in (1, 3, 5))

def omf_create_surface_data(obj_path_dir,colour_path):

    proj = omf.Project(
        name='Loop project',
        description='Bunch of surfaces'
    )

    onlyfiles = [f for f in listdir(obj_path_dir) if isfile(join(obj_path_dir, f))]

    all_sorts = pd.read_csv(os.path.join(colour_path, 'all_sorts_clean.csv'), ",")
    all_sorts=all_sorts.set_index('code')
    colour_map=open(obj_path_dir+'/loop_colour_map.clr','w')
    colour_map.write('{\tStart\tRed\tGreen\tBlue\t}\n')
    
    surfaces=[]
    for file in onlyfiles:
        if ('.obj' in file):

            obj=pd.read_csv(obj_path_dir+'/'+file,' ',names=["code","X","Y","Z"])
            indices=obj[obj['code']=='f']
            vertices=obj[obj['code']=='v']
            vertices=vertices.drop(['code'], axis=1)
            indices=indices[list("XYZ")].astype(int)
            i=indices.to_numpy()-1
            v=vertices.to_numpy()
            i=np.ascontiguousarray(i)
            v=np.ascontiguousarray(v)
            
            if('Fault_' in file or 'dtm' in file):
                rgb=255,255,255
            else:
                rgb=hextoints(all_sorts.loc[file.replace('.obj','')]['colour'])

            if(len(i)>0 and len(v)>0):
                surf = omf.SurfaceElement(
                    name=file.replace('.obj',''),
                    geometry=omf.SurfaceGeometry(
                        vertices=v,
                        triangles=i
                    ),
                    data=[
                        omf.ScalarData(
                            name='rand vert data',
                            array=np.random.rand(v.shape[0]),
                            location='vertices'
                        ),
                        omf.ScalarData(
                            name='rand face data',
                            array=np.random.rand(i.shape[0]),
                            location='faces'
                        )
                        ],           
                    color=[rgb[0],rgb[1],rgb[2]]
                    )
                surfaces.append(copy.deepcopy(surf))

    proj.elements = surfaces

    assert proj.validate()
    
    omf.OMFWriter(proj, obj_path_dir+'/loop_surf.omf')

            
obj_path_dir= './Example2/vtkleaflet_2021-07-16-09-43/'   # directory of existing obj surfaces
colour_path='./Example2/tmp/'
omf_create_surface_data(obj_path_dir,colour_path)


### GeoscienceAnalyst geohy5 format voxel model from LoopStructural  model

In [ ]:
#code to take a LoopStructural voxel model and save it out
#as a *.geoh5 GeoscienceAnalyst model 
#weird indexing because default LS block has X & Y swapped & Z -ve
#assumes model already created by LoopStructural, and minx, maxx info from main calculations
#Requires installation of https://github.com/MiraGeoscience/geoh5py 

# assumes LoopStructural model object has been calculated

from pathlib import Path
import numpy as np
from geoh5py.objects import BlockModel
from geoh5py.workspace import Workspace
 
voxel_size=500
sizex=int((maxx-minx)/voxel_size)
sizey=int((maxy-miny)/voxel_size)
sizez=int((model_top-model_base)/voxel_size)
nsteps=[sizex,sizey,sizez]

def create_geoh5_block_model_data(model,voxel_size,minx,miny,maxx,maxy,model_base,model_top,output_dir,nsteps):
    
    voxels=model.evaluate_model(model.regular_grid(nsteps=(nsteps[0],nsteps[1],nsteps[2]),shuffle=False),scale=False)

    name = "MyLoopBlockModel"

    # Generate a 3D array

    nodal_y = np.arange(0,maxx-minx+1,voxel_size)
    nodal_x = np.arange(0,maxy-miny+1,voxel_size)
    nodal_z = np.arange(model_top-model_base+1,0,-voxel_size)

    h5file_path = output_dir+"/loop_block_model.geoh5"


    # Create a workspace
    workspace = Workspace(h5file_path)

    grid = BlockModel.create(
        workspace,
        origin=[minx+(voxel_size/2), miny+(voxel_size/2), model_base+(voxel_size/2)],
        u_cell_delimiters=nodal_x,
        v_cell_delimiters=nodal_y,
        z_cell_delimiters=nodal_z,
        name=name,
        rotation=0,
        allow_move=False,
    )
    data = grid.add_data(
        {
            "DataValues": {
                "association": "CELL",
                "values": (
                    voxels.reshape((nodal_x.shape[0]-1,nodal_y.shape[0]-1,nodal_z.shape[0]-1)).transpose((1,0,2))
                ),
            }
        }
    )
    workspace.save_entity(grid)
    workspace.finalize()

output_dir= ''   # output directory to save geoh5 format voxel mdoel
create_geoh5_block_model_data(model,voxel_size,minx,miny,maxx,maxy,model_base,model_top,output_dir,nsteps)



### Open Mining Format omf format voxel model from LoopStructural  model

In [ ]:
#code to take a LoopStructural voxel model and save it out
#as a *.omf file that Micromine, Datamine, Vulcan... can read
#weird indexing because default LS block has X & Y swapped & Z -ve
# requires pip install omf (and omfvista and  ipyvtklink if you want to visualise them)
# https://omf.readthedocs.io/en/latest/content/examples.html
# https://gmggroup.org/projects/data-exchange-for-mine-software/
# https://github.com/gmggroup/omf
#assumes model already created by LoopStructural, and minx, maxx info from main calculations

# assumes LoopStructural model object has been calculated

voxel_size=500
sizex=int((maxx-minx)/voxel_size)
sizey=int((maxy-miny)/voxel_size)
sizez=int((model_top-model_base)/voxel_size)
nsteps=[sizex,sizey,sizez]

import omf
def create_omf_block_model_data(model,voxel_size,minx,miny,maxx,maxy,model_base,model_top,output_dir,colour_path,nsteps):
    
    voxels=model.evaluate_model(model.regular_grid(nsteps=(nsteps[0],nsteps[1],nsteps[2]),shuffle=False),scale=False)

    name = "MyLoopBlockModel"

    # Generate a 3D array
    proj = omf.Project(
        name='Loop project',
        description='Loop Block Model'
    )

    nodal_x = int((maxx-minx+1)/voxel_size)
    nodal_y = int((maxy-miny+1)/voxel_size)
    nodal_z = int((model_top-model_base+1)/voxel_size)
    print(nodal_x,nodal_y,nodal_z)
    print(minx,maxx,miny,maxy,model_base,model_top,voxel_size)
    vol = omf.VolumeElement(
        name='vol',
        geometry=omf.VolumeGridGeometry(
            axis_u=(1,0,0),
            axis_v=(0,1,0),
            axis_w=(0,0,-1),
            tensor_u=np.ones(nodal_y).astype(float)*voxel_size*(nodal_x/nodal_y),
            tensor_v=np.ones(nodal_x).astype(float)*voxel_size*(nodal_y/nodal_x),
            tensor_w=np.ones(nodal_z).astype(float)*voxel_size,
            origin=[minx-(voxel_size/2),miny-(voxel_size/2),model_base-(voxel_size/2)]
        ),
        data=[
            omf.ScalarData(
                name='LithoData',
                location='cells',
                array=voxels.reshape((nodal_z,nodal_x,nodal_y)).transpose((0,1,2)).flatten()
            )
        ]
    )

    proj.elements = [vol]

    assert proj.validate()
    
    omf.OMFWriter(proj, output_dir+'/loop_block.omf')

output_dir= ''   # output directory to save omf format voxel mdoel
colour_path='./Example2/tmp/'

create_omf_block_model_data(model,voxel_size,minx,miny,maxx,maxy,model_base,model_top,output_dir,colour_path,nsteps)

### Export model as gocad voxet

In [ ]:
voxel_size=500
sizex=int((maxx-minx)/voxel_size)
sizey=int((maxy-miny)/voxel_size)
sizez=int((model_top-model_base)/voxel_size)

nsteps=[sizex,sizey,sizez]
output_dir= ''   # output directory to save omf format voxel mdoel
file_name='loop_voxels'
data_label='lithoindex'


def write_vol_gocad(model, file_path,file_name, data_label, nsteps, real_coords=True):
    """
    Writes out the model as a 3d volume grid in GOCAD VOXET object format

    Parameters
    ----------
    model : GeologicalModel object
        Geological model to export
    file_name : string
        Name of file that model is exported to, including path, but without the file extension
    data_label : string
        A data label to insert into export file
    nsteps : np.array([num-x-steps, num-y-steps, num-z-steps])
        3d array dimensions

    Returns
    -------
    True if successful

    """
    # Define grid spacing in model scale coords
    loop_X = np.linspace(model.bounding_box[0, 0], model.bounding_box[1, 0], nsteps[0])
    loop_Y = np.linspace(model.bounding_box[0, 1], model.bounding_box[1, 1], nsteps[1])
    loop_Z = np.linspace(model.bounding_box[0, 2], model.bounding_box[1, 2], nsteps[2])

    # Generate model values in 3d grid
    xx, yy, zz = np.meshgrid(loop_X, loop_Y, loop_Z, indexing='ij')
    # xyz is N x 3 vector array
    xyz = np.array([xx.flatten(), yy.flatten(), zz.flatten()]).T
    vals = model.evaluate_model(xyz, scale=False)
    # Use FORTRAN style indexing for GOCAD VOXET
    vol_vals = np.reshape(vals, nsteps, order='F')
    bbox = model.bounding_box[:]
        
    # Convert bounding box to real world scale coords
    if real_coords:
        model.rescale(np.int64)
    print(type(vals[0]))
    # If integer values
    if (type(vals[0]) is np.int64 or type(vals[0]) is np.int32 ):
        d_type = np.int8
        no_data_val = None
        prop_esize = 1
        prop_storage_type = "Octet"

    # If float values
    elif type(vals[0]) is np.float32:
        d_type = np.dtype('>f4')
        no_data_val = -999999.0
        prop_esize = 4
        prop_storage_type = "Float"
    else:
        print("Cannot export volume to GOCAD VOXET file: Unsupported type {}".format(type(vals[0])))
        return False

    # Write out VOXET file
    vo_filename = file_name + ".vo"
    data_filename = file_name + "@@"
    try:
        with open(file_path+vo_filename, "w") as fp:
            fp.write("""GOCAD Voxet 1
HEADER {{
name: {name}
}}
AXIS_O 0.000000 0.000000 0.000000
AXIS_U 1.000000 0.000000 0.000000
AXIS_V 0.000000 1.000000 0.000000
AXIS_W 0.000000 0.000000 1.000000
AXIS_MIN {axismin1} {axismin2} {axismin3}
AXIS_MAX {axismax1} {axismax2} {axismax3}
AXIS_N {nsteps1} {nsteps2} {nsteps3}
AXIS_D {xdim} {ydim} {zdim}
AXIS_NAME "X" "Y" "Z"
AXIS_UNIT "m" "m" "m"
AXIS_TYPE even even even
PROPERTY 1 {propname}
PROPERTY_CLASS 1 {propname}
PROP_UNIT 1 {propname}
PROPERTY_CLASS_HEADER 1 {propname} {{
}}
PROPERTY_SUBCLASS 1 QUANTITY {prop_storage_type}
""".format(name=os.path.basename(file_name),
                nsteps1=nsteps[0], nsteps2=nsteps[1], nsteps3=nsteps[2],
                axismin1=bbox[0, 0], axismin2=bbox[0, 1], axismin3=bbox[0, 2],
                axismax1=bbox[1, 0], axismax2=bbox[1, 1], axismax3=bbox[1, 2],
                propname=data_label, prop_storage_type=prop_storage_type,
                xdim=(bbox[0, 0]-bbox[1, 0])/nsteps[0],
                ydim=(bbox[0, 1]-bbox[1, 1])/nsteps[1],
                zdim=(bbox[0, 2]-bbox[1, 2])/nsteps[2]))
            if no_data_val is not None:
                fp.write("PROP_NO_DATA_VALUE 1 {no_data_val}\n".format(no_data_val=no_data_val))
            fp.write("""PROP_ETYPE 1 IEEE
PROP_FORMAT 1 RAW
PROP_ESIZE 1 {prop_esize}
PROP_OFFSET 1 0
PROP_FILE 1 {prop_file}
END\n""".format(prop_file=data_filename, prop_esize=prop_esize))
    except IOError as exc:
        print("Cannot export volume to GOCAD VOXET file {}: {}".format(vo_filename, str(exc)))
        return False

    # Write out accompanying binary data file
    export_vals = np.array(vol_vals, dtype=d_type)
    try:
        with open(file_path+data_filename, "wb") as fp:
            export_vals.tofile(fp)
    except IOError as exc:
        print("Cannot export volume to GOCAD VOXET data file {}: {}".format(data_filename, str(exc)))
        return False
    return True

write_vol_gocad(model, output_dir,file_name, data_label, nsteps, real_coords=True)


## Extract all information from Loop gml file

### Extract Loop gml data from nodes as pandas

In [ ]:
# datatypes with topology
# ntype from   'fault' 
#              'formation' 
#              'group' 
#              'supergroup' 
# returns a pandas dataframe


import networkx as nx
import pandas as pd
def get_nodes_from_graph(Gloop,ntype):

    nodes_all=[]
    for v in Gloop.nodes():
        if(Gloop.nodes[v]['ntype']==ntype):
            nodes_all.append(v)
    nodes=Gloop.subgraph(nodes_all)
    data=pd.DataFrame.from_dict(dict(nodes.nodes(data=True)), orient='index')
    data['name']=data.index
    if(ntype=='fault'):
        columns={'ntype': 'ntype', 'Xmean': 'Xmean', 'Ymean': 'Ymean', 'Zmean': 'Zmean', 'HorizontalRadius': 'HzRad', 'VerticalRadius': 'Vrad', 'InfluenceDistance': 'NDist', 
             'IncLength': 'IncLength', 'f_colour': 'colour', 'Dip': 'Dip_1', 'DipDirection': 'DipDir', 'DipPolarity': 'Polarity', 
             'OrientationCluster': 'OCluster', 'LengthCluster': 'LCluster', 'ClosenessCentrality': 'CCentral', 'BetweennessCentrality':'BCentral'}
    elif(ntype=='formation'):
        columns={'ntype': 'ntype', 'label': 'label', 's_colour': 's_colour', 'group': 'group', 'StratType': 'StratType', 'uctype': 'uctype', 'GroupNum': 'GroupNumber', 
             'IndexInGp': 'IndexInGroup', 'NumberInGp': 'NumberInGroup', 'ThMedian': 'ThicknessMedian', 'ThStd': 'ThicknessStd', 'ThMethod': 'ThicknessMethod'}
    elif(ntype=='group' or ntype=='supergroup'):
        columns={'ntype': 'ntype', 'label': 'label', 's_colour': 's_colour', 'group': 'group', 'StratType': 'StratType', 'uctype': 'uctype', 'GroupNum': 'GroupNumber', 
             'IndexInGp': 'IndexInGroup', 'NumberInGp': 'NumberInGroup', 'ThMedian': 'ThicknessMedian', 'ThStd': 'ThicknessStd', 'ThMethod': 'ThicknessMethod'}
    data=data.rename(columns =columns, inplace = False)
    return(data)

Gloop=nx.read_gml('Example2/output/loop_colour.gml')
display(get_nodes_from_graph(Gloop,'fault'),
get_nodes_from_graph(Gloop,'formation'),
get_nodes_from_graph(Gloop,'supergroup'),
get_nodes_from_graph(Gloop,'group'))

### Extract geolocated point data from graph

In [ ]:
# datatypes with xyz but no topology
# dataype from 'fault_geom' 
#              'fault_displacement' 
#              'fault_strat_displacement' 
#              'contact' 
#              'orientation'
#              'raw_contact'
# returns a pandas dataframe

import networkx as nx
import pandas as pd
def get_point_data_from_graph(Gloop,datatype):

    nodes_all=[]
    for v in Gloop.nodes():
        if(Gloop.nodes[v]['ntype']=='points'):
            data=Gloop.nodes[v]['data']
    df = pd.DataFrame()
    if(datatype=='fault_geom'):
        Param1='Param1'
        Param2='Param2'
        Param3='Param3'
        Param4='Param4'
    elif(datatype=='fault_displacement'):
        Param1='HzRad'
        Param2='Vrad'
        Param3='NDist'
        Param4='Param4'
    elif(datatype=='fault_strat_displacement'):
        Param1='left_fm'
        Param2='right_fm'
        Param3='min_offset'
        Param4='strat_offset'
    elif(datatype=='contact'):
        Param1='Param1'
        Param2='Param2'
        Param3='Param3'
        Param4='Param4'
    elif(datatype=='orientation'):
        Param1='OCluster'
        Param2='LCluster'
        Param3='CCentral'
        Param4='BCentral'
    elif(datatype=='raw_contact'):
        Param1='group'
        Param2='angle'
        Param3='lsx'
        Param4='lsy'
        
    for d in data:
        if(d['type']==datatype):
            df = df.append({  'type': d['type'],
                              'name': d['name'],
                              'X': d['X'],
                              'Y': d['Y'],
                              'Z': d['Z'],
                              Param1:d['Param1'],
                              Param2:d['Param2'],
                              Param3:d['Param3'],
                              Param4:d['Param4']
                           }, ignore_index=True)
    return(df)

Gloop=nx.read_gml('Example2/output/loop.gml')
display(get_point_data_from_graph(Gloop,'fault_geom'),
       get_point_data_from_graph(Gloop,'fault_displacement'),
       get_point_data_from_graph(Gloop,'fault_strat_displacement'),
       get_point_data_from_graph(Gloop,'contact'),
       get_point_data_from_graph(Gloop,'orientation'),
       get_point_data_from_graph(Gloop,'raw_contact'))  


### Extract DTM grid

In [ ]:
# extract dtm grid
# returns a numpy array

import networkx as nx
import numpy as np
def get_dtm_from_graph(Gloop):

    nodes_all=[]
    for v in Gloop.nodes():
        if(Gloop.nodes[v]['ntype']=='dtm'):
            data=Gloop.nodes[v]['data']
            shape=Gloop.nodes[v]['shape']
            minx=Gloop.nodes[v]['minx']
            miny=Gloop.nodes[v]['miny']
            maxx=Gloop.nodes[v]['maxx']
            maxy=Gloop.nodes[v]['maxy']
            xscale=Gloop.nodes[v]['xscale']
            yscale=Gloop.nodes[v]['yscale']

            dtm=np.array(data.replace('[','').replace(']','').split(','), dtype=np.float32)
            shape=shape.replace('(','').replace(')','').split(',')
            x=int(shape[0])
            y=int(shape[1])
            dtm=dtm.reshape(x,y)
    return(minx,miny,maxx,maxy,xscale,yscale,dtm)        

Gloop=nx.read_gml('Example2/output/loop.gml')
display(get_dtm_from_graph(Gloop))


### extract bbox

In [ ]:
# extract bbox grid
# returns a numpy array

import networkx as nx
import numpy as np
def get_bbox_from_graph(Gloop):

    nodes_all=[]
    for v in Gloop.nodes():
        if(Gloop.nodes[v]['ntype']=='bbox'):
            data=Gloop.nodes[v]['data']
            bbox=np.array(data.replace('[','').replace(']','').split(','), dtype=np.float32)
    return(bbox)        

Gloop=nx.read_gml('Example2/output/loop.gml')
display(get_bbox_from_graph(Gloop))


### Extract crs

In [ ]:
# extract dst_crs grid
# returns a string

import networkx as nx
import numpy as np
def get_dst_crs_from_graph(Gloop):

    nodes_all=[]
    for v in Gloop.nodes():
        if(Gloop.nodes[v]['ntype']=='dst_crs'):
            data=Gloop.nodes[v]['data']
    return(data)        

Gloop=nx.read_gml('Example2/output/loop.gml')
display(get_dst_crs_from_graph(Gloop))

### Extract map2loop runtime metadata

In [ ]:
# extract dst_crs grid
# returns three strings

import networkx as nx
import numpy as np
def get_metadata_from_graph(Gloop):

    nodes_all=[]
    for v in Gloop.nodes():
        if(Gloop.nodes[v]['ntype']=='metadata'):
            run_flags=Gloop.nodes[v]['run_flags']
            c_l=Gloop.nodes[v]['c_l']
            config=Gloop.nodes[v]['config']
    return(c_l,run_flags,config)        

Gloop=nx.read_gml('Example2/output/loop.gml')
c_l,run_flags,config=get_metadata_from_graph(Gloop)
display("c_l",c_l,"run_flags",run_flags,"config",config)

### Extract subgraph based on node type

In [ ]:
# datatypes with xyz but no topology
# ntype from   'fault' 
#              'supergroup' 
#              'group' 
#              'formation' 
# returns a networkx graph

import networkx as nx
import pandas as pd

def get_subgraph_from_graph_nodes(Gloop,gtype):
    nodes_all=[]
    for v in Gloop.nodes():
        if(Gloop.nodes[v]['ntype']==gtype):
            nodes_all.append(v)
    subgraph=Gloop.subgraph(nodes_all)
    return(subgraph)

Gloop=nx.read_gml('Example2/output/loop.gml')
subgraph=get_subgraph_from_graph_nodes(Gloop,'fault')
nx.draw(subgraph,pos=nx.kamada_kawai_layout(subgraph),with_labels=True)

In [ ]:
subgraph=get_subgraph_from_graph_nodes(Gloop,'formation')
nx.draw(subgraph,pos=nx.kamada_kawai_layout(subgraph),with_labels=True)

In [ ]:
subgraph=get_subgraph_from_graph_nodes(Gloop,'group')
nx.draw(subgraph,pos=nx.kamada_kawai_layout(subgraph),with_labels=True)

In [ ]:
subgraph=get_subgraph_from_graph_nodes(Gloop,'supergroup')
nx.draw(subgraph,pos=nx.kamada_kawai_layout(subgraph),with_labels=True)

### Extract subgraph based on edge type

In [ ]:
# datatypes with topology
# etype from   'fault_fault' 
#              'fault_group' 
#              'group_formation' 
#              'supergroup_group' 
#              'group_group'
# returns a networkx graph

import networkx as nx
import pandas as pd

def get_subgraph_from_graph_edges(Gloop,etype):
    edges_all=[]
    for v in Gloop.edges:
        if(Gloop.edges[v]['etype']==etype):
            edges_all.append(v)
    subgraph=nx.Graph().to_directed()
    for e in edges_all:
        subgraph.add_edge(e[0],e[1])
    return(subgraph)

Gloop=nx.read_gml('Example2/output/loop.gml')
subgraph=get_subgraph_from_graph_edges(Gloop,'fault_fault')
nx.draw(subgraph,pos=nx.kamada_kawai_layout(subgraph),with_labels=True)

In [ ]:
subgraph=get_subgraph_from_graph_edges(Gloop,'fault_group')
nx.draw(subgraph,pos=nx.kamada_kawai_layout(subgraph),with_labels=True)

In [ ]:
subgraph=get_subgraph_from_graph_edges(Gloop,'group_formation')
nx.draw(subgraph,pos=nx.kamada_kawai_layout(subgraph),with_labels=True)

In [ ]:
subgraph=get_subgraph_from_graph_edges(Gloop,'supergroup_group')
nx.draw(subgraph,pos=nx.kamada_kawai_layout(subgraph),with_labels=True)

In [ ]:
subgraph=get_subgraph_from_graph_edges(Gloop,'group_group')
nx.draw(subgraph,pos=nx.kamada_kawai_layout(subgraph),with_labels=True)